In [1]:
import numpy as np
import pandas as pd
import sklearn
import mglearn
from sklearnex  import  patch_sklearn 
patch_sklearn ()


dataset = pd.read_csv('dataset_limpio.csv')
dataset = dataset.drop(['Unnamed: 0'], axis=1)

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
from sklearnex  import  patch_sklearn 
patch_sklearn ()
from time import time
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.decomposition import PCA

reduced_data = PCA(n_components = 0.95).fit_transform(dataset)
print(reduced_data.shape)
def bench_marks(algoritm, name):
    
    t0 = time()
    estimator = make_pipeline(StandardScaler(), algoritm).fit(reduced_data)
    labels= estimator[-1].labels_
    
    fit_time = time() - t0
    results = [name, fit_time, estimator[-1].inertia_]
    results += []

    results += [silhouette_score(reduced_data, labels, metric = 'euclidean')]
    results += [davies_bouldin_score(reduced_data,labels)]
    results += [calinski_harabasz_score(reduced_data, labels)]
    
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)
    results += [n_clusters]
    
    formatter_result = (
        "{:8s}\t\t{:.3f}s\t{:.0f}\t{:.3f}\t\t{:.3f}\t\t{:.2f}\t\t{:.0f}"
    )
    print(formatter_result.format(*results))

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


(343880, 2)


In [ ]:
from sklearn.cluster import KMeans
from sklearnex  import  patch_sklearn 
patch_sklearn ()


print(82 * "_")
print("init\t\t\ttime\tinertia\t\tsilhouette\tdavies_bouldin\tcalinski_harabasz\tn_clusters\n")

n_clusters = [2, 3, 4, 5]
random_state = np.random.seed(42)

for i in n_clusters:
    
#     #kmeans
    kmeans = KMeans(init="random", n_clusters= 2, random_state=random_state)
    bench_marks(kmeans, name="random")
    kmeans = KMeans(init="k-means++", n_clusters= 2, random_state=random_state)
    bench_marks(kmeans, name="k-means++")

    
print(82 * "_")

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


__________________________________________________________________________________
init			time	inertia		silhouette	davies_bouldin	calinski_harabasz	n_clusters

(343880,)
